<a href="https://colab.research.google.com/github/OgunSerifOnargan/arkitek_notebooks/blob/main/yolov8_cls_moving_average(need_maintenance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from IPython import display
import os
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# değiştirilecek path'ler
isim = 'uygar'
drive_raw_videos_path = f"/content/drive/MyDrive/classify/raw_videos/blood/{isim}/preprocess" # predict edilecek videoların dosya yolu
output_drive = f'/content/drive/MyDrive/classify/colab_test_videos/blood/{isim}'  # predict edilmiş videoların toplanacağı drive dosya yolu
colab_videos_path = "/content/videos/"

In [ ]:
os.makedirs("/content/output_videos/",exist_ok=True)
os.makedirs('/content/models/', exist_ok=True)

In [ ]:
import shutil
model_path = '/content/models/'
shutil.copy('/content/drive/MyDrive/classify/models/LATEST_blood_v6.pt',model_path)

'/content/models/LATEST_blood_v6.pt'

In [ ]:
shutil.copy('/content/drive/MyDrive/classify/raw_videos/blood/uygar/finished/midsommar_1.mp4','/content/videos')

'/content/videos/midsommar_1.mp4'

In [ ]:
!pip install ultralytics

display.clear_output()

In [ ]:
# @title
!pip install yt_dlp
import yt_dlp
def download_playlist(playlist_url, output_folder):
    ydl_opts = {
        'format': 'best',
        'quiet': False,
        'outtmpl': os.path.join(output_folder, '%(playlist_index)s - %(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([playlist_url])
        except:
             pass

def download_video(video_url, output_folder):
    ydl_opts = {
        'format': 'best',
        'quiet': False,
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([video_url])
            print("Video indirme tamamlandı.")
        except Exception as e:
            print(f'Hata oluştu: {str(e)}')

def rename_file_replace_char(file_path, old_char_list, new_char):
    filename = os.path.basename(file_path)
    for old_char in old_char_list:
        filename = filename.replace(old_char, new_char)

    new_file_path = os.path.join(os.path.dirname(file_path), filename)
    os.rename(file_path, new_file_path)



display.clear_output()


## **Playlist Downloader**

In [ ]:
!pip install yt_dlp
os.makedirs(colab_videos_path, exist_ok = True)
display.clear_output()

In [ ]:
url = r'https://youtube.com/playlist?list=PL00nAaYo53pgWTxSo3peJlEYI5WS95WA-&si=wn-fIrqfeScPAoUX'
download_playlist(url,colab_videos_path)
display.clear_output()
print('Success!')

# **Drive Videos**

In [ ]:
# shutil.rmtree('/content/videos')

In [ ]:
import shutil
shutil.copytree(drive_raw_videos_path,colab_videos_path)

'/content/videos/'

In [ ]:
# videos klasörünün içindeki isimleri güncelleme
videos_path="/content/videos/"
if not os.path.exists(videos_path):
        os.makedirs(videos_path)
oldchars = ['!','：','｜','⧸','？',' ','"','#','$','%','&','(',')',' ','*','+',',','-','/',':',';''<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~']

for root,dirs,files in os.walk(videos_path):
    for file in files:
        video_path = os.path.join(root,file)
        rename_file_replace_char(old_char_list=oldchars,new_char='',file_path=video_path)
        SOURCE_VIDEO_PATH=video_path
        file_name = file.split('/')[0]
        print(file_name)

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm

# Load the YOLOv8 model
model = YOLO(r'/content/models/LATEST_blood_v6.pt')

# Define the number of frames for the moving average
moving_average_frames = 10  # You can adjust this value as needed

# Create a list to store the frames for the moving average
frame_buffer = []

for root, dirs, files in os.walk(videos_path):
    for file in files:

        # Open the video file
        video_path = os.path.join(root, file)
        cap = cv2.VideoCapture(video_path)
        video_name = file.split('.')[0]

        # Video dosyasının çözünürlüğü ve fps değeri
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        fps = 30  # Örnek olarak 30 FPS kullanıldı, uygun bir değer seçmelisiniz

        # VideoWriter objesini oluşturun
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec'i, kullanılabilir diğer codec'leri de deneyebilirsiniz
        out = cv2.VideoWriter(f'/content/output_videos/{video_name}.mp4', fourcc, fps, (frame_width, frame_height))

        cap = cv2.VideoCapture(video_path)

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Create a tqdm progress bar
        progress_bar = tqdm(total=total_frames, unit="frames")

        # Loop through the video frames
        while cap.isOpened():
            # Read a frame from the video
            success, frame = cap.read()

            if success:
                # Append the frame to the frame buffer
                frame_buffer.append(frame)

                # If the buffer size exceeds the specified moving_average_frames, calculate the moving average
                if len(frame_buffer) > moving_average_frames:
                    frame_buffer.pop(0)  # Remove the oldest frame

                # Calculate the moving average frame
                moving_average_frame = np.mean(frame_buffer, axis=0).astype(np.uint8)

                # Run YOLOv8 inference on the moving average frame
                results = model(moving_average_frame, verbose=False)

                # Visualize the results on the moving average frame
                annotated_frame = results[0].plot()

                # Write the annotated frame to the output video
                out.write(annotated_frame)

                # Update the progress bar
                progress_bar.update(1)

                # Display the annotated frame
                # cv2.imshow("YOLOv8 Inference", annotated_frame)

                # Break the loop if 'q' is pressed
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
            else:
                # Break the loop if the end of the video is reached
                break

        # Release the video capture and writer objects, and close the display window
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        #
        shutil.copy(f'/content/output_videos/{video_name}.mp4', f"{output_drive}/{video_name}_mv10.mp4")
        progress_bar.close()


  0%|          | 0/5833 [00:00<?, ?frames/s]Downloading https://ultralytics.com/assets/Arial.ttf to '/root/.config/Ultralytics/Arial.ttf'...

100%|██████████| 755k/755k [00:00<00:00, 18.8MB/s]
100%|██████████| 5833/5833 [22:55<00:00,  4.24frames/s]


In [ ]:
shutil.copy("/content/output_videos/1ThisiswhatIthoughtwhileskydivingmp4.mp4","/content/drive/MyDrive/colab_test_videos/height")

'/content/drive/MyDrive/colab_test_videos/height/1ThisiswhatIthoughtwhileskydivingmp4.mp4'